In [1]:
import math
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("shots_data.csv")
df

,team,x,y,fgmade
0,Team A,-5.0,26.4,0
1,Team A,-0.8,1.2,1
2,Team A,-13.9,9.5,1
3,Team A,-5.4,26.2,0
4,Team A,-4.9,14.5,1
...,...,...,...,...
499,Team B,-0.5,2.2,0
500,Team B,3.5,7.6,0
501,Team B,0.2,8.8,0
502,Team B,-0.6,0.7,1


In [24]:
x = df["x"].values
y = df["y"].values
distance = (x*x + y*y)**(.5)
df["distance"] = distance


In [28]:
def shotType(df):
    if (df["y"] <= 7.8) and (df["distance"] > 22):
        return "C3"
    elif (df["distance"] > 23.75):
        return "NC3"
    else:
        return "2PT"

In [30]:
df["shot"] = df.apply(shotType, axis=1)
df

,team,x,y,fgmade,distance,shot
0,Team A,-5.0,26.4,0,26.869313,NC3
1,Team A,-0.8,1.2,1,1.442221,2PT
2,Team A,-13.9,9.5,1,16.836270,2PT
3,Team A,-5.4,26.2,0,26.750701,NC3
4,Team A,-4.9,14.5,1,15.305555,2PT
...,...,...,...,...,...,...
499,Team B,-0.5,2.2,0,2.256103,2PT
500,Team B,3.5,7.6,0,8.367198,2PT
501,Team B,0.2,8.8,0,8.802272,2PT
502,Team B,-0.6,0.7,1,0.921954,2PT


In [65]:
def efg(df, team):
    shots = (df.groupby(["team","shot"]).count()["fgmade"]).loc[team]
    made = (df.groupby(["team","shot"]).sum(numeric_only = True)["fgmade"]).loc[team]
    twos = (made.loc["2PT"]) / (shots.loc["2PT"])
    NC3 = (1.5*(made.loc["NC3"])) / (shots.loc["NC3"])
    C3 = (1.5*(made.loc["C3"])) / (shots.loc["C3"])
    
    
    return(twos, NC3, C3)

print(efg(df, "Team A"))
print(efg(df, "Team B"))


(0.4896907216494845, 0.4632352941176471, 0.75)
(0.44666666666666666, 0.5080645161290323, 0.5)


In [60]:
def shotDist(df, team):
    attempted = (df.groupby(["team"]).count()["fgmade"]).loc[team]
    C3 = (df.groupby(["team","shot"]).sum(numeric_only = True)["fgmade"]).loc[team].loc["C3"]
    C3 = (C3/attempted)
    NC3 = (df.groupby(["team","shot"]).sum(numeric_only = True)["fgmade"]).loc[team].loc["NC3"]
    NC3 = (NC3/attempted)
    twos = (df.groupby(["team","shot"]).sum(numeric_only = True)["fgmade"]).loc[team].loc["2PT"]
    twos = (twos/attempted)
    
    return(C3, NC3, twos)

A = shotDist(df, "Team A")
B = shotDist(df, "Team B")

print("Team A distribution - Corner 3: " + str(A[0]) + ", Non-Corner 3: " + str(A[1]) + ", Two-Pointers: " + str(A[2]))
print("Team B distribution - Corner 3: " + str(B[0]) + ", Non-Corner 3: " + str(B[1]) + ", Two-Pointers: " + str(B[2]))

Team A distribution - Corner 3: 0.03214285714285714, Non-Corner 3: 0.075, Two-Pointers: 0.3392857142857143
Team B distribution - Corner 3: 0.017857142857142856, Non-Corner 3: 0.09375, Two-Pointers: 0.29910714285714285
